In [40]:
from gensim.models import Word2Vec
import helper
import pandas as pd
import csv 
from nltk.cluster import KMeansClusterer
import nltk
from sklearn import cluster
from sklearn import metrics

In [8]:
### Handle input

trainingdata = pd.read_csv("../datasets/train/SemEval2018-T3-train-taskA_emoji.txt", delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)


In [9]:
corpus = helper.allbags(trainingdata['Tweet text'])

In [10]:
### Word2Vec maker

model = Word2Vec(corpus, min_count=1, size=200, window=8)
model.save("word2vec.model")

In [184]:
words = model.wv.index2word
X = model.wv[words]

In [41]:
print (model.wv.most_similar(positive=['want'], negative=[], topn=2))

[('not', 0.999947726726532), ('but', 0.9999476671218872)]


In [153]:
### Gebruik sklearn want multithread

kmeans = cluster.KMeans(n_clusters=2000, precompute_distances=True)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)

Cluster id labels for inputted data
[  21   30   82 ... 1813 1031 1343]
Centroids data
[[ 2.1002023e-03 -1.2687952e-03 -7.7084568e-04 ...  2.6867106e-03
  -2.6423070e-03 -1.4337866e-02]
 [ 1.0610403e-01 -3.3780660e-02 -3.8619321e-02 ...  9.3790352e-02
  -7.7377401e-02 -5.1984781e-01]
 [ 1.9172736e-01 -7.1447432e-02 -7.6589674e-02 ...  1.6962774e-01
  -1.4249974e-01 -9.6515870e-01]
 ...
 [ 3.3360100e-03 -1.9953428e-03 -2.5526755e-03 ...  2.7567763e-03
  -4.5482945e-03 -1.9183930e-02]
 [ 3.2486720e-03 -5.4055161e-04 -6.5637671e-04 ...  1.3784224e-03
  -2.2951849e-03 -1.4492221e-02]
 [ 1.3852923e-03 -1.9383514e-03  2.5885447e-04 ... -3.8292119e-04
   1.4443498e-03 -3.0640010e-03]]
Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):
-4.0117197
Silhouette_score: 
-0.011113764


In [157]:
len(centroids)

2000

In [42]:
kmeans.predict([model.wv['dog'],model['haha']])

NameError: name 'kmeans' is not defined

In [43]:
inputbag = helper.allbags(["Hi my name is jeff", "I think you're cool"])
resulting_clusters = []

for row in inputbag:
    clusters_row = []
    filtered_words = list(filter(lambda x: x in words, row))
    clusters_row.append(list(kmeans.predict(model[filtered_words])))
    resulting_clusters.append(clusters_row)
            
resulting_clusters

NameError: name 'words' is not defined

In [189]:
filtered_words

[]

In [5]:
from sklearn.externals import joblib

## Save the classifier
# _ = joblib.dump(kmeans, "kmeansclass")

In [19]:
### Load the classifier
kmeansmodel = joblib.load("kmeansclass.model")
w2vmodel = Word2Vec.load("word2vec.model")
vocab = w2vmodel.wv.index2word

inputbag = helper.allbags(["Hi my name is jeff", "I think you're cool"])
resulting_clusters = []

for row in inputbag:
    filtered_words = list(filter(lambda x: x in vocab, row))
    resulting_clusters.append(list(kmeansmodel.predict(model[filtered_words])))
            
resulting_clusters

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


[[107, 1264, 1346, 1264], [1264, 330, 1478, 1346]]

In [35]:

onehotvec = np.zeros((len(resulting_clusters), 2000))
for i in range(len(resulting_clusters)):
    onehotvec[i, resulting_clusters[i]] = 1


7